In [1]:
import os
import numpy as np
import pandas as pd
from utils.utils import *
from utils.kabsch2D import *
from nsd_access import NSDAccess 
from scipy import stats

True


In [2]:
nsda = NSDAccess(nsd_dir)

In [114]:
def create_rotation_df(subj, rois, sess):
    cols = ['source', 'base', 'target', 'U', 'error']
    rotations = np.zeros(((len(rois) * (len(rois)-1)) * 2 +1, len(cols)), dtype=object)
    i = 0 
    randoms = []
    for roi_source in rois.keys():
        mds_source = np.load(f'{proj_dir}/rsa/mds/{subj}/{subj}_{sess}_{roi_source}_mds.npy', allow_pickle=False)
        for roi_target in rois.keys():
            roi = roi_target
            if roi_source != roi_target:
                for j in range(3):
                    mds_target = np.load(f'{proj_dir}/rsa/mds/{subj}/{subj}_{sess}_{roi}_mds.npy', allow_pickle=False)
                    target_shuffle = mds_target
                    # flip the target when j is at 1, does nothing when j = 0 
                    rotations[i, 1] = roi_target
                    if j == 1:
                        mds_target = np.dot(mds_target, np.array([[-1, 0], [0, 1]]))
                        roi_target = roi_target + "_flipped"
                    
                    if j == 2:
                        permut_indices = np.random.permutation(len(mds_source))   # seems to be working more consistently than shuffle
                        mds_shuffle= mds_source[permut_indices]
                        permut_indices = np.random.permutation(len(mds_target))
                        target_shuffle = mds_target[permut_indices]
                        U_s, t_s = kabsch2D(mds_shuffle, target_shuffle, translate=True)
                        rotated_source_s = rotate(mds_shuffle, U_s)
                        randoms.append(avg_distance(rotated_source_s, target_shuffle, t_s))
                        del mds_shuffle, target_shuffle, rotated_source_s
                        continue
                    U, t= kabsch2D(mds_source, mds_target, translate=True )
                    rotations[i, 0] = roi_source
                    rotations[i, 2] = roi_target
                    rotations[i, 3] = U
                    rotated_source = rotate(mds_source, U)
                    rotations[i, 4] = avg_distance(rotated_source, mds_target, t)
                    i += 1
  
    rotations[i, 0] = "random"
    rotations[i, 1] = "random"
    rotations[i, 2] = "random"
    
    rotations[i, 4] = np.mean(randoms)
    df = pd.DataFrame(rotations, columns=cols)
    df = df.astype({'error': 'float32'}) # needed for later indexing
    return df

# check difference between distances with shuffle and not shuffle 
# if there's is lower overall distance, +1 
# check if the difference between ROI actually significant 
# signed ranked test
# https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test

In [115]:
def get_ranking(df):
    # get the index of the lowest error for each pair in each ROI
    indx_to_keep = df.groupby(['source', 'base'])['error'].idxmin()
    df_filtered = df.loc[indx_to_keep]
    rank = df_filtered.groupby('source')['error'].mean().sort_values()
    return rank

https://www.sciencedirect.com/science/article/pii/S0167715298000066
https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.weightedtau.html

Its cool but doesn't give a p value

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kendalltau.html#scipy.stats.kendalltau 
Normal Kendall Tau might be the thing?? 

I asked chatGPT, and it tells me to use Friedman test, by looking at the difference between the average rank

In [116]:
rotations_df = create_rotation_df(subj=subj_list[0], rois=rois, sess=sessions[0])
rotations_df

264


,source,base,target,U,error
0,V1,V2,V2,"[[-0.2742083, 0.9616703], [-0.9616703, -0.2742...",0.598965
1,V1,V2,V2_flipped,"[[0.5983935837897106, -0.8012022958524938], [0...",0.809925
2,V1,V3,V3,"[[-0.19200626, 0.9813937], [-0.9813937, -0.192...",0.761273
3,V1,V3,V3_flipped,"[[-0.9242557639758863, 0.3817738633764112], [-...",0.541289
4,V1,hV4,hV4,"[[-0.70612174, 0.7080905], [-0.7080905, -0.706...",0.670393
...,...,...,...,...,...
260,TO-2,LO-2,LO-2,"[[0.49966773, 0.8662172], [-0.8662172, 0.49966...",0.713642
261,TO-2,LO-2,LO-2_flipped,"[[0.385028965325518, 0.9229044890238433], [-0....",0.712025
262,TO-2,TO-1,TO-1,"[[0.28874344, -0.9574065], [0.9574065, 0.28874...",0.730878
263,TO-2,TO-1,TO-1_flipped,"[[-0.9972822034670352, -0.07367636424210111], ...",0.763123


In [117]:
rois

{'V1': 1,
 'V2': 2,
 'V3': 3,
 'hV4': 4,
 'VO-1': 5,
 'VO-2': 6,
 'PHC-1': 7,
 'PHC-2': 8,
 'LO-1': 9,
 'LO-2': 10,
 'TO-1': 11,
 'TO-2': 12}

In [118]:
def get_rank_dict(subj_list, rois, sessions):
    dict, dict_values, dict_matrices = {}, {}, {}
    for i in range(len(subj_list)):
        rotations_df = create_rotation_df(subj=subj_list[i], rois=rois, sess=sessions[i])
        rank = get_ranking(rotations_df)
     #   rank_int = [rois[roi] for roi in rank.index]
        #dict[subj_list[i]] = rank_int
        dict_values[subj_list[i]] = rank

    return dict_values

In [119]:
subj_list = [f'subj0{i}' for i in range(1,9)]
#sessions = [37, 37, 29, 27, 37, 29, 37, 27]

rank_dict = get_rank_dict(subj_list=subj_list, rois=rois, sessions=sessions)
rank_dict

264
264
264
264
264
264
264
264


{'subj01': source
 VO-2      0.655353
 hV4       0.663349
 PHC-2     0.663958
 V2        0.667335
 VO-1      0.672721
 LO-1      0.679328
 PHC-1     0.679637
 V1        0.684180
 V3        0.687971
 TO-2      0.708574
 LO-2      0.719411
 TO-1      0.746349
 random    0.867267
 Name: error, dtype: float32,
 'subj02': source
 VO-1      0.652863
 LO-1      0.659404
 TO-2      0.670224
 PHC-2     0.680714
 TO-1      0.685626
 V3        0.688547
 LO-2      0.688816
 VO-2      0.691446
 V1        0.691903
 V2        0.710392
 hV4       0.713045
 PHC-1     0.753166
 random    0.877615
 Name: error, dtype: float32,
 'subj03': source
 PHC-1     0.570285
 VO-2      0.580243
 VO-1      0.582375
 TO-2      0.604287
 hV4       0.610812
 V3        0.614896
 V1        0.627387
 LO-1      0.629560
 V2        0.646641
 PHC-2     0.648145
 LO-2      0.665605
 TO-1      0.686909
 random    0.872389
 Name: error, dtype: float32,
 'subj04': source
 PHC-1     0.613973
 PHC-2     0.617088
 LO-1      0.63412

In [121]:
rank_dict_values_norm = {key : (value - value.min()) / (value.max() - value.min()) for key, value in rank_dict.items()}
rank_dict_values_norm

{'subj01': source
 VO-2      0.000000
 hV4       0.037734
 PHC-2     0.040606
 V2        0.056543
 VO-1      0.081958
 LO-1      0.113137
 PHC-1     0.114596
 V1        0.136036
 V3        0.153924
 TO-2      0.251147
 LO-2      0.302282
 TO-1      0.429403
 random    1.000000
 Name: error, dtype: float32,
 'subj02': source
 VO-1      0.000000
 LO-1      0.029102
 TO-2      0.077245
 PHC-2     0.123920
 TO-1      0.145772
 V3        0.158770
 LO-2      0.159967
 VO-2      0.171668
 V1        0.173701
 V2        0.255965
 hV4       0.267769
 PHC-1     0.446285
 random    1.000000
 Name: error, dtype: float32,
 'subj03': source
 PHC-1     0.000000
 VO-2      0.032963
 VO-1      0.040018
 TO-2      0.112552
 hV4       0.134150
 V3        0.147670
 V1        0.189013
 LO-1      0.196207
 V2        0.252749
 PHC-2     0.257725
 LO-2      0.315522
 TO-1      0.386039
 random    1.000000
 Name: error, dtype: float32,
 'subj04': source
 PHC-1     0.000000
 PHC-2     0.012021
 LO-1      0.07774

In [127]:
rank_total = np.zeros(len(rois) +1)
for i in range(len(subj_list)):
    rank_total += list(rank_dict.values())[i]
rank_total

source
VO-2      5.134792
hV4       5.271054
PHC-2     5.289833
V2        5.461472
VO-1      5.046467
LO-1      5.280855
PHC-1     5.270800
V1        5.395471
V3        5.242516
TO-2      5.310410
LO-2      5.399963
TO-1      5.597116
random    6.963881
Name: error, dtype: float64

In [45]:
rank_dict_values.values()

dict_values([source
VO-2     0.655353
hV4      0.663349
PHC-2    0.663958
V2       0.667335
VO-1     0.672721
LO-1     0.679328
PHC-1    0.679637
V1       0.684180
V3       0.687971
TO-2     0.708574
LO-2     0.719411
TO-1     0.746349
Name: error, dtype: float32, source
VO-1     0.652863
LO-1     0.659404
TO-2     0.670224
PHC-2    0.680714
TO-1     0.685626
V3       0.688547
LO-2     0.688816
VO-2     0.691446
V1       0.691903
V2       0.710392
hV4      0.713045
PHC-1    0.753166
Name: error, dtype: float32, source
PHC-1    0.570285
VO-2     0.580243
VO-1     0.582375
TO-2     0.604287
hV4      0.610812
V3       0.614896
V1       0.627387
LO-1     0.629560
V2       0.646641
PHC-2    0.648145
LO-2     0.665605
TO-1     0.686909
Name: error, dtype: float32, source
PHC-1    0.613973
PHC-2    0.617088
LO-1     0.634121
LO-2     0.640148
VO-1     0.640737
TO-2     0.648957
V2       0.652842
VO-2     0.654249
TO-1     0.654271
V1       0.659262
hV4      0.664817
V3       0.677680
Name: er

In [9]:
corr = np.zeros((len(subj_list), len(subj_list)))
p_values = np.zeros((len(subj_list), len(subj_list)))

for i in range(len(subj_list)):
    for j in range(len(subj_list)):
        res = stats.kendalltau(list(rank_dict.values())[i], list(rank_dict.values())[j])
        corr[i, j] = res.correlation
        p_values[i, j] = round(res.pvalue, 5)
        

In [10]:
corr_df = pd.DataFrame(corr, columns=rank_dict.keys(), index=rank_dict.keys())
corr_df

,subj01,subj02,subj03,subj04,subj05,subj06,subj07,subj08
subj01,1.000000,-0.090909,0.060606,-0.363636,-0.333333,-0.121212,0.696970,0.333333
subj02,-0.090909,1.000000,0.000000,-0.060606,0.333333,0.424242,-0.090909,-0.030303
subj03,0.060606,0.000000,1.000000,-0.212121,-0.181818,-0.393939,0.060606,-0.060606
subj04,-0.363636,-0.060606,-0.212121,1.000000,-0.060606,0.151515,-0.424242,-0.303030
subj05,-0.333333,0.333333,-0.181818,-0.060606,1.000000,0.181818,-0.333333,-0.272727
subj06,-0.121212,0.424242,-0.393939,0.151515,0.181818,1.000000,-0.242424,0.121212
subj07,0.696970,-0.090909,0.060606,-0.424242,-0.333333,-0.242424,1.000000,0.333333
subj08,0.333333,-0.030303,-0.060606,-0.303030,-0.272727,0.121212,0.333333,1.000000


In [11]:
pvalues_df = pd.DataFrame(p_values, columns=rank_dict.keys(), index=rank_dict.keys())
pvalues_df

,subj01,subj02,subj03,subj04,subj05,subj06,subj07,subj08
subj01,0.00000,0.73731,0.84059,0.11595,0.15259,0.63836,0.00097,0.15259
subj02,0.73731,0.00000,1.00000,0.84059,0.15259,0.06287,0.73731,0.94656
subj03,0.84059,1.00000,0.00000,0.38070,0.45902,0.08632,0.84059,0.84059
subj04,0.11595,0.84059,0.38070,0.00000,0.84059,0.54520,0.06287,0.19695
subj05,0.15259,0.15259,0.45902,0.84059,0.00000,0.45902,0.15259,0.24958
subj06,0.63836,0.06287,0.08632,0.54520,0.45902,0.00000,0.31081,0.63836
subj07,0.00097,0.73731,0.84059,0.06287,0.15259,0.31081,0.00000,0.15259
subj08,0.15259,0.94656,0.84059,0.19695,0.24958,0.63836,0.15259,0.00000


In [126]:
rank_total.sort_values()    # add average when randomly rodated

source
VO-1      5.046467
VO-2      5.134792
V3        5.242516
PHC-1     5.270800
hV4       5.271054
LO-1      5.280855
PHC-2     5.289833
TO-2      5.310410
V1        5.395471
LO-2      5.399963
V2        5.461472
TO-1      5.597116
random    6.963881
Name: error, dtype: float64

In [20]:
0.84 * 8

6.72

In [15]:
rank_total_norm = np.zeros(len(rois))
for i in range(len(subj_list)):
    rank_total_norm += list(rank_dict_values_norm.values())[i]

rank_total_norm.sort_values()

# Use random (shuffled) as 1 


source
VO-1     0.994154
VO-2     2.067867
PHC-1    2.636121
PHC-2    2.866173
TO-2     2.915441
LO-1     3.185457
V3       3.197119
hV4      3.387463
V1       4.135132
LO-2     4.319080
V2       4.568211
TO-1     6.106050
Name: error, dtype: float64

In [16]:
proj_dir

'/media/harveylab/STORAGE1 nA/stan-thesis/projects/NSD'

In [16]:
# apply rotation on one subject
def apply_rotation(target, subj, rois, sessions):
    # obtain the rotation matrix again
    rotations_df = create_rotation_df(subj, rois, sessions)
    rotations_df = rotations_df[rotations_df['base'] == target]
    indx_to_keep = rotations_df.groupby(['source', 'base'])['error'].idxmin()
    rotations_df = rotations_df.loc[indx_to_keep]
    rotations_df = rotations_df.reset_index(drop=True)
    print(rotations_df.U)
    rotated_path = os.path.join(proj_dir, 'MDS_rotated', subj)
    print(rotated_path)
    if not os.path.exists(rotated_path):
        os.makedirs(rotated_path)
        print(os.path.exists(rotated_path))
    for roi in rois.keys():
        source_mds = np.load(f'{proj_dir}/rsa/mds/{subj}/{subj}_{sessions}_{roi}_mds.npy', allow_pickle=False)
        mds_rotated_file = os.path.join(rotated_path, f'{subj}_{sessions}_{roi}_mds_rotated_{target}.npy')
        # if the source is our reference point, just save it without applying rotation
        if roi == target:
            if not os.path.exists(mds_rotated_file):
                np.save(mds_rotated_file, source_mds, allow_pickle=True)
            continue 
        U = rotations_df.loc[rotations_df['source'] == roi, 'U'].squeeze()
        mds_rotated = np.dot(source_mds, U)  # apply rotation with dot product
        if not os.path.exists(mds_rotated_file):
            np.save(mds_rotated_file, mds_rotated, allow_pickle=True)
    
    

In [19]:
for i in range(1, 9):
    apply_rotation('VO-1', f'subj0{i}', rois, sessions[i-1])


0     [[0.7962450919381348, -0.6049741759483885], [0...
1     [[0.2885183, -0.9574744], [0.9574744, 0.2885183]]
2     [[-0.13038644, -0.9914632], [0.9914632, -0.130...
3     [[0.61425931469044, -0.7891042353935447], [0.7...
4     [[-0.8189188787935667, -0.5739092872183608], [...
5       [[0.9274614, -0.373919], [0.373919, 0.9274614]]
6     [[-0.86375904, -0.5039052], [0.5039052, -0.863...
7     [[-0.19596773, 0.9806104], [-0.9806104, -0.195...
8     [[0.5480861313583374, -0.83642189869267], [0.8...
9     [[-0.40075022, 0.9161874], [-0.9161874, -0.400...
10    [[0.3004219, 0.95380646], [-0.95380646, 0.3004...
Name: U, dtype: object
/media/harveylab/STORAGE1 nA/stan-thesis/projects/NSD/MDS_rotated/subj01
0     [[0.7591267, 0.6509428], [-0.6509428, 0.7591267]]
1     [[-0.005775407097485141, -0.9999833221973546],...
2     [[0.5831776, 0.8123447], [-0.8123447, 0.5831776]]
3     [[0.37653014, 0.92640436], [-0.92640436, 0.376...
4     [[0.0014738211015534186, -0.9999989139250904],...
5     [[